In [ ]:
%cd /ibex/user/slimhy/PADS/code
import numpy as np
import os

PARTS_DIR = "/ibex/project/c2273/PADS/3DCoMPaT_occ/parts"
SAMPLES_DIR = "/ibex/project/c2273/PADS/3DCoMPaT_occ/samples"
MAX_PART_DROP = 16

def load_part_bbs(model_id):
    """
    Load part bounding boxes for a given model ID.
    """
    bb_file = os.path.join(SAMPLES_DIR, f"{model_id}_orig_0_bbs.pkl")
    # Read the file
    bb_data = np.load(bb_file, allow_pickle=True)
    bb_data = {k:v for k,v in bb_data}
    bb_data = {k:np.array(v.vertices) for k,v in bb_data.items()}
    return bb_data # {part_key: [N_p, 3]}  N_p = 131072


def load_occs(model_id, part_drop_id=None):
    """
    Load queries and occupancies for a given model ID.
    
    Args:
        model_id (str): The model identifier
        part_drop_id (int, optional): The part drop identifier. If None, loads the original 
                                    version with no parts dropped.
    
    Returns:
        tuple: (queries, occupancies)
            - queries: array of shape [5, N_p, 3]
            - occupancies: array of shape [5, N_p, 1] squeezed to [5, N_p]
    """
    if part_drop_id is None:
        # Load original version (no parts dropped)
        occs = os.path.join(SAMPLES_DIR, f"{model_id}_orig_0_occs.npy")
        queries = os.path.join(SAMPLES_DIR, f"{model_id}_orig_0_points.npy")
    else:
        # Load part-drop version
        occs = os.path.join(SAMPLES_DIR, f"{model_id}_part_drop_{part_drop_id}_occs.npy")
        queries = os.path.join(SAMPLES_DIR, f"{model_id}_part_drop_{part_drop_id}_points.npy")
    
    # Read the files
    occs = np.load(occs)
    queries = np.load(queries)
    
    return queries, occs.reshape(*queries.shape[:2], -1).squeeze()


def load_part_surf_points(model_id):
    """
    Load part surface points for a given model ID.
    """
    part_file = os.path.join(PARTS_DIR, f"{model_id}.npy")
    # Read the file
    part_data = np.load(os.path.join(PARTS_DIR, part_file), allow_pickle=True)
    part_data = part_data.item()
    part_data = {k:np.array(v).squeeze() for k,v in part_data.items()}
    return part_data # [N_p, 3]  N_p = 131072


def get_dropped_part_key(model_id, part_drop_id):
    """
    Get the key for the dropped part.
    """
    bb_file = os.path.join(SAMPLES_DIR, f"{model_id}_part_drop_{part_drop_id}_bbs.pkl")
    # Read the file
    bb_data = np.load(bb_file, allow_pickle=True)
    bb_data = {k:v for k,v in bb_data}
    
    # Read original part labels
    bb_file = os.path.join(SAMPLES_DIR, f"{model_id}_orig_0_bbs.pkl")
    # Read the file
    bb_data_orig = np.load(bb_file, allow_pickle=True)
    bb_data_orig = {k:v for k,v in bb_data_orig}
    
    # Find the dropped part
    dropped_part_key = set(bb_data_orig.keys()) - set(bb_data.keys())
    assert len(dropped_part_key) == 1
    return list(dropped_part_key)[0]

/ibex/user/slimhy/PADS/code


In [22]:
part_points = load_part_surf_points("28_0ce")
part_points.keys()

part_bbs = load_part_bbs("28_0ce")
part_bbs.keys()

queries, occs = load_occs("28_0ce", 0)

dropped_key = get_dropped_part_key("28_0ce", 2)

In [19]:
part_points[(0, 157)].shape

(131072, 3)

In [20]:
part_bbs[(0, 157)].shape

(8, 3)

In [21]:
queries.shape, occs.shape

((5, 131072, 3), (5, 131072))